## Controle de fatura

Este artigo fornece uma visão geral da tabela de uso faturável do sistema, incluindo o esquema e exemplos de consultas. Com as tabelas do sistema, os dados de uso faturável da sua conta são centralizados e roteados para todas as regiões, permitindo que você visualize o uso global da sua conta de qualquer região em que seu espaço de trabalho esteja localizado.

Para obter informações sobre como usar esta tabela para monitorar custos e exemplos de consultas, consulte Monitorar custos usando tabelas do sistema.

Caminho da tabela: Esta tabela do sistema está localizada em system.billing.usage.

## Exemplo de código

In [0]:
%sql
SELECT
  identity_metadata.run_as,
  sku_name,
  cloud,
  usage_date,
  usage_quantity
FROM
  system.billing.usage
WHERE
  usage_unit = 'DBU'

## Quantidade DBU usada por pessoa

In [0]:
%sql
SELECT
  identity_metadata.run_as AS usuario,
  billing_origin_product AS servico, --Tipo de serviço
  SUM(usage_quantity) AS total_usage_qtty,
  usage_unit
FROM
  system.billing.usage
GROUP BY
  1, 2, 4
ORDER BY
  total_usage_qtty DESC

## Quantidade DBU usada por pessoa no ultimo mês

In [0]:
%sql
SELECT
  identity_metadata.run_as AS usuario,
  billing_origin_product AS servico,
  SUM(usage_quantity) AS total_dbu
FROM
  system.billing.usage
WHERE
  usage_unit = 'DBU'
  AND usage_date >= current_date() - interval 30 days
GROUP BY
  1, 2
ORDER BY
  total_dbu DESC

## Custos

In [0]:
dbutils.widgets.text("DBU_COST", "0.40")
dbutils.widgets.text("DSU_COST", "0.023")
dbutils.widgets.text("DSU_GB_AWS", "0.09")

In [0]:
%sql
SELECT
  identity_metadata.run_as AS usuario,
  billing_origin_product AS servico,
  usage_unit,
  SUM(usage_quantity) AS total_usage_qtty,
  ROUND(
    SUM(
      CASE 
        WHEN usage_unit = 'DBU' THEN usage_quantity * ${DBU_COST}
        WHEN usage_unit = 'DSU' THEN usage_quantity * ${DSU_COST}
        WHEN usage_unit = 'GB'  THEN usage_quantity * ${DSU_GB_AWS}
        ELSE usage_quantity 
      END
    ), 2
  ) AS total_cost
FROM
  system.billing.usage
GROUP BY
  1, 2, 3
ORDER BY
  total_cost DESC